In [117]:
# !wget https://snap.stanford.edu/data/cit-HepTh-abstracts.tar.gz 
# !tar -xf cit-HepTh-abstracts.tar.gz 
# !wget https://raw.githubusercontent.com/TeaDove/arxiv-journal-parsing/master/data/abrs.csv 
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_1999.csv
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_2000.csv
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_2001.csv
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_2002.csv
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_2003.csv
# !wget https://github.com/TeaDove/arxiv-journal-parsing/raw/master/data/scimagojr_2004.csv

### Ссылки:
- Датасет: https://snap.stanford.edu/data/cit-HepTh.html
- Датасет с журналами: https://www.scimagojr.com/journalrank.php
- Абривиатура в оригинал: https://images.webofknowledge.com/images/help/WOS/0-9_abrvjt.html


In [58]:
from pathlib import Path
from tqdm.notebook import tqdm

import pandas as pd

In [4]:
year = 1992
abstart_files = []
has_journal = 0
count = 0
existing_journals = []
while year < 2004:
  for file in list(Path(str(year)).iterdir()):
    count += 1
    abs = open(file, 'r').read()
    idx = abs.find('Journal-ref: ')
    if idx != -1:
      has_journal+=1

      journals = abs[idx+13: abs.find('\\', idx)-1].split(';')
      journals = map(lambda x: ''.join(x.split()[:-3]).split(',')[0], journals)
      
      for jounral in journals:
        if jounral not in existing_journals:
          existing_journals.append(jounral)
  year += 1

print(len(existing_journals), has_journal, count, has_journal / count)

412 21022 29555 0.711284046692607


In [69]:
abrs = pd.read_csv("abrs.csv", skipinitialspace=True, header=0, index_col=0)
abrs.head()

,abr,issn
0,modphyslett,"02177323, 17936632"
1,jgeomphys,03930440
2,nuclphys,03759474
3,physrev,"24699934, 24699926"
4,intjmodphys,"0217751X, 1793656X"


In [107]:
year = 1992
id_year_issn = []

while year < 2004:
  for file in list(Path(str(year)).iterdir()):
    abs = open(file, 'r').read()
    
    idx = abs.find('Journal-ref: ')
    issn = 0
    if idx != -1:
      journals = abs[idx+13: abs.find('\\', idx)-1].split(';')
      journals = map(lambda x: ''.join(x.split()[:-3]).split(',')[0].lower().replace('.', ''), journals)
      
      for jounral in journals:
        issn_tmp = abrs.loc[abrs.abr == jounral].issn.values

        if len(issn_tmp) != 0:
           issn = issn_tmp[0]
    id_year_issn.append([file.name.split('.')[0], year+1, issn])

    
  year += 1
paper_to_journal = pd.DataFrame(id_year_issn, columns=['idx', 'year_of_publ', 'journal_issn']) 
paper_to_journal.head()

,idx,year_of_publ,journal_issn
0,9208008,1993,"14320916, 00103616"
1,9202022,1993,0
2,9206105,1993,"10897658, 00222488"
3,9210128,1993,00319163
4,9209115,1993,"02177323, 17936632"


In [123]:
journals_df = []
for i in range(6):
  df = pd.read_csv(f'scimagojr_{i + 1999}.csv', delimiter=';', header=0)
  journals_df.append(df)
journals_df[2].head()

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2001),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories
0,1,16801,Annual Review of Biochemistry,journal,"15454509, 00664154","39,946",Q1,293,23,88,4150,4121,88,"32,25","180,43",United States,Northern America,Annual Reviews Inc.,"1946-1948, 1950-1960, 1962-2020",Biochemistry (Q1)
1,2,20651,Annual Review of Immunology,journal,"15453278, 07320582","36,369",Q1,301,24,84,4102,4422,84,"47,97","170,92",United States,Northern America,Annual Reviews Inc.,1983-2020,Immunology (Q1); Immunology and Allergy (Q1)
2,3,18395,Annual Review of Cell and Developmental Biology,journal,"15308995, 10810706","31,364",Q1,220,24,66,4054,2061,63,"27,69","168,92",United States,Northern America,Annual Reviews Inc.,1995-2019,Cell Biology (Q1); Developmental Biology (Q1)
3,4,18434,Cell,journal,"00928674, 10974172","29,247",Q1,776,357,1120,15274,33005,1111,"28,80","42,78",United States,Northern America,Cell Press,1974-2020,"Biochemistry, Genetics and Molecular Biology (..."
4,5,14181,Annual Review of Neuroscience,journal,"0147006X, 15454126","20,785",Q1,241,41,62,6572,1757,62,"27,50","160,29",United States,Northern America,Annual Reviews Inc.,1978-2020,Neuroscience (miscellaneous) (Q1)


In [165]:
new_journals_df = []
for journal_df_year in tqdm(journals_df):
  new_journal_year = []
  for row in tqdm(list(journal_df_year.iterrows())):
    issns = row[1].Issn.split(', ')
    if len(issns) != 2:
       issns = [issns[0], None]
    new_journal_year.append([row[1]['Cites / Doc. (2years)'], row[1]['SJR Best Quartile'], row[1]['H index'], *issns])
  new_journals_df.append(pd.DataFrame(new_journal_year, columns=['impact', 'quartile', 'h_index', 'issn1', 'issn2']))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/17123 [00:00<?, ?it/s]

  0%|          | 0/17503 [00:00<?, ?it/s]

  0%|          | 0/18218 [00:00<?, ?it/s]

  0%|          | 0/19382 [00:00<?, ?it/s]

  0%|          | 0/20031 [00:00<?, ?it/s]

  0%|          | 0/20833 [00:00<?, ?it/s]

In [195]:
papers_journal_df = []
count = 0
all = 0
for row in tqdm(list(paper_to_journal.iterrows())):
  all += 1
  year = row[1].year_of_publ
  year = year if year > 1998 else 1999
  journals_year = new_journals_df[1999 - year]
  issns = str(row[1].journal_issn).split(', ')
  to_df = [1, 1, 'Q4']

  if issns[0] != '0':
    journals = []
    for issn in issns:
      journals.append(journals_year.loc[(journals_year.issn1 == issn) | (journals_year.issn2 == issn)])
    
    real_journal = [journal for journal in journals if len(journal.values) != 0]
    if real_journal:
      real_journal = real_journal[0]
      to_df = [real_journal.impact.values[0], real_journal.h_index.values[0], real_journal.quartile.values[0]]

  papers_journal_df.append([row[1].idx, *to_df])

papers_journal_df = pd.DataFrame(papers_journal_df, columns=['idx', 'impact', 'h_index', 'quartile'])
papers_journal_df.head()

  0%|          | 0/29555 [00:00<?, ?it/s]

,idx,impact,h_index,quartile
0,9208008,"1,57",152,Q1
1,9202022,1,1,Q4
2,9206105,"0,93",119,Q2
3,9210128,1,1,Q4
4,9209115,"1,08",88,Q2


In [198]:
papers_journal_df.to_csv("papers_journal.csv")